In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sqlalchemy import create_engine
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
db_name = "timescale"
db_user = "admin"
db_password = "TimescalePassword"
db_host = "localhost"
db_port = "5433"
database_uri = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(database_uri)
df = pd.read_sql("SELECT * FROM filteredsensordatagrid", engine)

sections = ["Vihannekset", "Hedelmät", "Viljatuotteet", "Einekset", "Lihatuotteet", "Kuivatuotteet", "Välipalat", "Virvokkeet", "Maitotuotteet", "WC-paperit", "Pesuaineet", "Hygieniatuotteet", "Makeiset", "Pakasteet", "Lemmikkien ruuat", "Pulloautomaatti"]


In [2]:
title = widgets.HTML(
    value="<h2 style='font-size:20px'>logistic regression</h2>",
)
osasto_X_dropdown = widgets.Dropdown(options=sections, description="osasto X:")
osasto_Y_dropdown = widgets.Dropdown(options=sections, description="osasto Y:")
button = widgets.Button(description="laske Tn")
output = widgets.Output()

def nappi(b):
    with output:
        clear_output(wait=True)
        osasto_X = osasto_X_dropdown.value
        osasto_Y = osasto_Y_dropdown.value
        
        X = df[[osasto_X]]  # jos käyt osastolla x...
        y = df[osasto_Y]    # target

        #datan jako
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        #regressio
        model = LogisticRegression()
        model.fit(X_train, y_train)

        # ennustus
        prob = model.predict_proba([[1]])[0, 1]  # Tn että käyt osastolla Y jos käyt osastolla X
        print(f"TN sille, että käyt osastolla '{osasto_Y}' jos käyt osastolla '{osasto_X}': [{prob:.2f}]")

        # AUC
        tn_y = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, tn_y)
        print(f"\nAUC Score: [{auc:.2f}]")
        print("\nAUC = 0.5: sama kuin arvaus\nAUC 0.6 - 0.7: heikko\nAUC 0.7 - 0.8: OK\nAUC 0.8 - 0.9: hyvä\nAUC 0.9 - 1.0: erinomainen")


button.on_click(nappi)
display(title, osasto_X_dropdown, osasto_Y_dropdown, button, output)

HTML(value="<h2 style='font-size:20px'>logistic regression</h2>")

Dropdown(description='osasto X:', options=('Vihannekset', 'Hedelmät', 'Viljatuotteet', 'Einekset', 'Lihatuotte…

Dropdown(description='osasto Y:', options=('Vihannekset', 'Hedelmät', 'Viljatuotteet', 'Einekset', 'Lihatuotte…

Button(description='laske Tn', style=ButtonStyle())

Output()